<a href="https://colab.research.google.com/github/c0nrrrs/Connor_DTSC3020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [15]:
(1) #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [13]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [16]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [19]:
# Q1 — Write your answer here
# --- Q1 Robust Solution ---

from io import StringIO

def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML."""
    # Wrap HTML string for read_html (avoids the FutureWarning)
    tables = pd.read_html(StringIO(html))
    df = None
    for t in tables:
        if t.shape[1] >= 3:
            df = t
            break
    df = flatten_headers(df)
    return df


def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable)."""
    # Normalize headers
    df.columns = [c.strip().title() for c in df.columns]

    # Handle possible variants like "Numeric", "Numeric Code"
    rename_map = {
        "Alpha-2 Code": "Alpha-2 Code",
        "Alpha-3 Code": "Alpha-3 Code",
        "Country": "Country",
        "Numeric": "Numeric Code",
        "Numeric Code": "Numeric Code"
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    # Ensure required columns exist
    keep = [c for c in ["Country", "Alpha-2 Code", "Alpha-3 Code", "Numeric Code"] if c in df.columns]
    df = df[keep].copy()

    # Clean up text
    for c in ["Country", "Alpha-2 Code", "Alpha-3 Code"]:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip().str.upper()

    # Convert numeric
    if "Numeric Code" in df.columns:
        df["Numeric Code"] = pd.to_numeric(df["Numeric Code"], errors="coerce").astype("Int64")

    return df.dropna(subset=["Country"])


def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric Code and return Top-N."""
    if "Numeric Code" in df.columns:
        df = df.sort_values(by="Numeric Code", ascending=False)
    return df.head(top)


# --- Run Q1 ---
html = fetch_html("https://www.iban.com/country-codes")
df_q1 = q1_read_table(html)
df_q1 = q1_clean(df_q1)
top15_q1 = q1_sort_top(df_q1)

print("Top-15 by Numeric Code:")
print(top15_q1)

df_q1.to_csv("data_q1.csv", index=False)



Top-15 by Numeric Code:
                                               Country Alpha-2 Code  \
247                                             ZAMBIA           ZM   
246                                              YEMEN           YE   
192                                              SAMOA           WS   
244                                  WALLIS AND FUTUNA           WF   
240                 VENEZUELA (BOLIVARIAN REPUBLIC OF)           VE   
238                                         UZBEKISTAN           UZ   
237                                            URUGUAY           UY   
35                                        BURKINA FASO           BF   
243                              VIRGIN ISLANDS (U.S.)           VI   
236                     UNITED STATES OF AMERICA (THE)           US   
219                       TANZANIA, UNITED REPUBLIC OF           TZ   
108                                        ISLE OF MAN           IM   
113                                             JERSE

## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [20]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [21]:
# Q2 — Write your answer here
# --- Q2 Solution ---

def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns: rank, title, link, points, comments, user (optional)."""
    soup = BeautifulSoup(html, "lxml")
    items = soup.select(".athing")
    data = []

    for item in items:
        rank_tag = item.select_one(".rank")
        title_tag = item.select_one(".titleline a")
        subtext = item.find_next_sibling("tr").select_one(".subtext")

        rank = rank_tag.text.strip().replace(".", "") if rank_tag else "0"
        title = title_tag.text.strip() if title_tag else ""
        link = title_tag.get("href", "") if title_tag else ""

        points_tag = subtext.select_one(".score") if subtext else None
        user_tag = subtext.select_one(".hnuser") if subtext else None
        comments_tag = subtext.find_all("a")[-1] if subtext and subtext.find_all("a") else None

        points = points_tag.text.strip() if points_tag else "0 points"
        user = user_tag.text.strip() if user_tag else ""
        comments = comments_tag.text.strip() if comments_tag else "0 comments"

        data.append({
            "rank": rank,
            "title": title,
            "link": link,
            "points": points,
            "comments": comments,
            "user": user
        })

    return pd.DataFrame(data)


def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values."""
    def extract_int(x):
        m = re.search(r"(\d+)", str(x))
        return int(m.group(1)) if m else 0

    for col in ["rank", "points", "comments"]:
        df[col] = df[col].apply(extract_int)

    # Fill missing text fields
    for col in ["title", "link", "user"]:
        df[col] = df[col].fillna("").astype(str).str.strip()

    return df


def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N."""
    return df.sort_values(by="points", ascending=False).head(top)


# Q2 — Write your answer here
html = fetch_html("https://news.ycombinator.com/")
df_q2 = q2_parse_items(html)
df_q2 = q2_clean(df_q2)
top15_q2 = q2_sort_top(df_q2)

print("Top-15 by Points:")
print(top15_q2[["rank", "title", "points", "comments", "user"]])

df_q2.to_csv("data_q2.csv", index=False)





Top-15 by Points:
    rank                                              title  points  comments  \
29    30                   Solarpunk is happening in Africa    1097       533   
28    29                          End of Japanese community     871       675   
9     10                             Ratatui – App Showcase     673       189   
7      8            FBI tries to unmask owner of archive.is     410       221   
2      3             ICC ditches Microsoft 365 for openDesk     323        90   
0      1  Kimi K2 Thinking, a SOTA open-source trillion-...     320       104   
3      4  Open Source Implementation of Apple's Private ...     292        53   
18    19  Cloudflare Tells U.S. Govt That Foreign Site B...     247       148   
20    21  IKEA launches new smart home range with 21 Mat...     206       163   
17    18  Australia has so much solar that it's offering...     204       153   
26    27  I may have found a way to spot U.S. at-sea str...     203       262   
14    15  